# Agent-based model using `Agents.jl`
Simon Frost (@sdwfrost), 2020-04-27

## Introduction

The agent-based model approach, implemented using [`Agents.jl`](https://github.com/JuliaDynamics/Agents.jl) taken here is:

- Stochastic
- Discrete in time
- Discrete in state

## Libraries

In [ ]:
using Agents
using Random
using DataFrames
using Distributions
using DrWatson
using StatsPlots
using BenchmarkTools

## Utility functions

In [ ]:
function rate_to_proportion(r::Float64,t::Float64)
    1-exp(-r*t)
end;

## Transitions

First, we have to define our agent, which has an `id`, and a `status` (`:S`,`:I`, or `:R`).

In [ ]:
mutable struct Person <: AbstractAgent
    id::Int64
    status::Symbol
end

This utility function sets up the model, by setting parameter fields and adding agents to the model.

In [ ]:
function init_model(β::Float64,c::Float64,γ::Float64,N::Int64,I0::Int64)
    properties = @dict(β,c,γ)
    model = ABM(Person; properties=properties)
    for i in 1:N
        if i <= I0
            s = :I
        else
            s = :S
        end
        p = Person(i,s)
        p = add_agent!(p,model)
    end
    return model
end;

The following function applies a series of functions to each agent.

In [ ]:
function agent_step!(agent, model)
    transmit!(agent, model)
    recover!(agent, model)
end;

This is the transmission function; note that it operates on susceptibles making contact, rather than being focused on infected. This is an inefficient way of doing things, but shows the parallels between the different implementations.

In [ ]:
function transmit!(agent, model)
    # If I'm not susceptible, I return
    agent.status != :S && return
    ncontacts = rand(Poisson(model.properties[:c]))
    for i in 1:ncontacts
        # Choose random individual
        alter = random_agent(model)
        if alter.status == :I && (rand() ≤ model.properties[:β])
            # An infection occurs
            agent.status = :I
            break
        end
    end
end;

This is the recovery function.

In [ ]:
function recover!(agent, model)
    agent.status != :I && return
    if rand() ≤ model.properties[:γ]
            agent.status = :R
    end
end;

We need some reporting functions.

In [ ]:
susceptible(x) = count(i == :S for i in x)
infected(x) = count(i == :I for i in x)
recovered(x) = count(i == :R for i in x);

## Time domain

In [ ]:
δt = 0.1
nsteps = 400
tf = nsteps*δt
t = 0:δt:tf;

## Parameter values

In [ ]:
β = 0.05
c = 10.0*δt
γ = rate_to_proportion(0.25,δt);

## Initial conditions

In [ ]:
N = 1000
I0 = 10;

## Random number seed

In [ ]:
Random.seed!(1234);

## Running the model

In [ ]:
abm_model = init_model(β,c,γ,N,I0)

In [ ]:
to_collect = [(:status, f) for f in (susceptible, infected, recovered)]
abm_data, _ = run!(abm_model, agent_step!, nsteps; adata = to_collect);

## Post-processing

In [ ]:
abm_data[!,:t] = t;

## Plotting

In [ ]:
plot(t,abm_data[:,2],label="S",xlab="Time",ylabel="Number")
plot!(t,abm_data[:,3],label="I")
plot!(t,abm_data[:,4],label="R")

## Benchmarking

In [ ]:
@benchmark begin
abm_model = init_model(β,c,γ,N,I0)
abm_data, _ = run!(abm_model, agent_step!, nsteps; adata = to_collect)
end